In [1]:
from imp import reload
from hitbtc import HitBTC, OrderBook
import model

import pandas as pd
import torch
import json
import uuid
#_ = reload(model), reload(hitbtc)

In [2]:
class Trader(HitBTC):
    def __init__(self, key, secret, pair, tick_size=0.001):
        self.pair = pair
        self.tick_size = tick_size
        super(Trader, self).__init__(key, secret)
        self.send('subscribeTrades', {'symbol': pair}, 0)
        self.lstm = model.get_model()
        self.last_buy_id = None
        self.last_sell_id = None
        
        self.num_correct = 1
        self.num_incorrect = 0
        self.next_prediction = None
        self.last_value = None
        self.error_diff_long = 0
        self.error_diff_short = 0
        self.log = []
    
    def handle_messages(self, message):
        self.log.append(f'correct: {self.num_correct}, incorrect: {self.num_incorrect}: {self.num_correct * 100 / (self.num_incorrect + self.num_correct)}%, error long: {trader.error_diff_long}, error short: {trader.error_diff_short}')
        
        new_value = None
        if message.get('method', '') == 'updateTrades':
            new_value = float(json.loads(message['updateTrades'])['params']['data'][0]['price'])
            
        if new_value is not None:
            if self.next_prediction is not None and self.last_value is not None:
                if new_value > self.last_value:
                    if self.next_prediction == 1:
                        self.num_correct += 1
                    else:
                        self.num_incorrect += 1
                        self.error_diff_long += abs(new_value - self.last_value)
                elif new_value < self.last_value:
                    if self.next_prediction == 0:
                        self.num_correct += 1
                    else:
                        self.num_incorrect += 1
                        self.error_diff_short += abs(new_value - self.last_value)
            self.last_value = new_value

            if len(self.messages) == self.max_size:
                df = pd.DataFrame([a for b in [json.loads(m)['params']['data'] for m in self.messages] for a in b])
                df = df[df.id > 1]
                df = df.assign(price=lambda x: pd.to_numeric(x.price)).assign(relative_returns=lambda x: (x.price - x.price.shift(1)) / x.price)[-10:]
                inputs = torch.from_numpy(pd.np.expand_dims(df.relative_returns.values, 1)[None]).float()
                predictions = self.lstm(inputs)
                prediction = torch.max(predictions, 1)[1][0].item()
                self.next_prediction = prediction

                if prediction == 1:
                    buy_price = self.last_value - 1 * self.tick_size
                    sell_price = self.last_value + 10 * self.tick_size
                else:
                    buy_price = self.last_value - 10 * self.tick_size
                    sell_price = self.last_value + 1 * self.tick_size

                if self.last_buy_id:
                    self.log.append(f'Cancelling buy order')
                    self.send('cancelOrder', {'clientOrderId': self.last_buy_id}, id=0)
                if self.last_sell_id:
                    self.log.append(f'Cancelling sell order')
                    self.send('cancelOrder', {'clientOrderId': self.last_sell_id}, id=1)

                self.log.append(f'Creating new buy ({buy_price}) and sell ({sell_price}) orders')
                self.last_buy_id, self.last_sell_id = uuid.uuid1().hex, uuid.uuid1().hex
                self.send('newOrder', {'clientOrderId': self.last_buy_id,
                                       'symbol': self.pair,
                                       'side': 'buy',
                                       'price': str(buy_price),
                                       'quantity': '0.001'}, id=0)

                self.send('newOrder', {'clientOrderId': self.last_sell_id,
                                       'symbol': self.pair,
                                       'side': 'sell',
                                       'price': str(sell_price),
                                       'quantity': '0.001'}, id=1)


In [3]:
key = 'd4e97b7f4dda4163fff6860f4db03646'
secret = 'f59e82ee2d2bd4506dc9906422097db5'

client = HitBTC(key, secret)
trader = Trader(key, secret, 'ETHUSD')
book = OrderBook(key, secret, 'ETHUSD')

In [4]:
from IPython.display import clear_output

In [ ]:
import time
while True:
    clear_output(True)
    for l in trader.log: print(l)
    time.sleep(0.5)
    

correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error long: 0, error short: 0
correct: 1, incorrect: 0: 100.0%, error 

In [12]:
#client.send('unsubscribeTicker', {'symbol': 'ETHBTC'})
#client.send('unsubscribeTrades', {'symbol': 'ETHBTC'})

In [18]:
client.send('getTradingBalance', {}, 1)

In [19]:
pd.DataFrame([json.loads(m) for m in client.messages])

,id,jsonrpc,result
0,15276309459475,2.0,True
1,1,2.0,"[{'currency': '1ST', 'available': '0', 'reserv..."
